In [553]:
import sys
import pandas as pd
import numpy as np
import math
sys.path.append('/Users/victorkaplan/Desktop/HackRice/HackRice11-OrderManagement/src/Database')
import database

In [ ]:
#CURRENT VERSION****************

In [654]:
from IPython.display import clear_output

In [739]:
def readin():
    workersdb = database.get_under_directory('/Workers')
    workers = pd.DataFrame.from_dict(workersdb, orient='index')
    workers = workers.reset_index()
    workers = workers.drop('index', axis=1)
    workers.head()
    
    equipdb = database.get_under_directory('/Equipments')
    equip = pd.DataFrame.from_dict(equipdb, orient='index')
    equip = equip.reset_index()
    equip = equip.drop('index', axis=1)
    equip.head()
    
    facildb = database.get_under_directory('/Facilities')
    facil = pd.DataFrame.from_dict(facildb, orient='index')
    facil = facil.reset_index()
    facil['workersIn']= facil['workersIn'].astype(int)
    facil['Maximum Occupacy']= facil['Maximum Occupacy'].astype(int)
    facil = facil.drop('index', axis=1)
    facil.head()
    
    workOrdersdb = database.get_under_directory('/WorkOrders')
    workOrders = pd.DataFrame.from_dict(workOrdersdb, orient='index')
    workOrders = workOrders.reset_index()
    workOrders['Priority(1-5)']= workOrders['Priority(1-5)'].astype(int)
    workOrders['Submission Timestamp'] = pd.to_datetime(workOrders['Submission Timestamp'])
    workOrders = workOrders.drop('index', axis=1)
    workOrders.head()
    return workers, equip, facil, workOrders

read = readin()
workers = read[0]
equip= read[1]
facil= read[2]
workOrders= read[3]

def priority(workOrderdf):
    realPriors = workOrderdf['Priority(1-5)']
    times = workOrderdf['Submission Timestamp']
    maxtime = max(times)
    timePrior = ([-(time - maxtime).days for time in times])
    final = (timePrior) + (realPriors)
    workOrderdf['newPrior'] = final
    return workOrderdf

workOrders = priority(workOrders)
workOrders.head()

,Equipment ID,Equipment Type,Facility,Priority(1-5),Submission Timestamp,Time to Complete,Work Order,done,inProgress,newPrior,timeLeft
0,P032,Pump,Fac1,5,2018-09-14 22:12:00,3,1001,False,False,6,0
1,Con391,Conveyer,Fac3,1,2018-09-13 18:30:00,9,1002,False,False,4,0
2,Sep028,Seperator,Fac4,2,2018-09-13 05:41:00,3,1003,False,False,5,0
3,Sen826,Sensor,Fac5,4,2018-09-13 10:20:00,1,1004,False,False,7,0
4,Sec032,Security,Fac1,1,2018-09-13 10:20:00,2,1005,False,False,4,0


In [740]:
def dist(facilityloc1, facilityloc2):
    if facilityloc1 == facilityloc2:
        return 0
    return 2

def bid(workerdf, equipType, facility, timeToComplete, shift):
    df = workerdf.copy()
    df = df.loc[df['inTask'] == 'False']
    df = df.loc[df['Shifts'] == shift]
    df = df[df['Equipment Certification(s)'].str.contains(equipType,case=False)]
    final = []
    for i in df.index:
        final.append((i, dist(df.at[i, 'Loc'], facility) + int(timeToComplete)))
    return final

def pair(workerdf, workOrderdf, facilitydf, shift):
    sliceddf = workOrderdf.loc[workOrderdf['inProgress'] == 'False']
    currentJob = sliceddf['newPrior'].idxmax()
    print(list(sliceddf['newPrior']))
    print(currentJob)
    
    if int(facilitydf.loc[facilitydf['Facility'] == (workOrderdf.at[currentJob, 'Facility']), 'Maximum Occupacy']) <= int(facilitydf.loc[facilitydf['Facility'] == (workOrderdf.at[currentJob, 'Facility']), 'workersIn']):
        print('full')
        workOrderdf.at[currentJob, 'newPrior'] -= 1
        return None
    
    bids = bid(workerdf, workOrderdf.at[currentJob, 'Equipment Type'], workOrderdf.at[currentJob, 'Facility'], workOrderdf.at[currentJob, 'Time to Complete'], shift)    
    lowest = math.inf
    lowestPair = None
    for bidPair in bids:
        if bidPair[1] < lowest:
            lowest = bidPair[1]
            lowestPair = bidPair
    if lowestPair != None:
        return (workerdf.at[lowestPair[0], 'Name'], workOrderdf.at[currentJob, 'Work Order ']), (lowestPair[0],currentJob), lowest
    workOrderdf.at[currentJob, 'newPrior'] -= 1
    return None

def update(pair, workerdf, workOrderdf, time, facilitydf):
    oldloc = list((facilitydf.loc[facilitydf['Facility'] == workerdf.at[pair[0], 'Loc']])['Facility'])[0]
    newloc = list((facilitydf.loc[facilitydf['Facility'] == workOrderdf.at[pair[1], 'Facility']])['Facility'])[0]
    oldlocinx = list(facilitydf.loc[facilitydf['Facility'] == oldloc].index)[0]
    newlocinx = list(facilitydf.loc[facilitydf['Facility'] == newloc].index)[0]
    
    facilitydf.at[oldlocinx, 'workersIn'] -= 1
    facilitydf.at[newlocinx, 'workersIn'] += 1
    
    workOrderdf.at[pair[1], 'inProgress'] = True
    workOrderdf.at[pair[1], 'timeLeft'] = time
    
    workerdf.at[pair[0], 'inTask'] = True
    workerdf.at[pair[0], 'Loc'] = newloc
    workerdf.at[pair[0], 'TasktimeLeft'] = time

def assignAll(workerdf, workOrderdf, facilitydf, shift):
    for i in range(len(workOrderdf.index)*2):
#         print(workerdf['inTast'])
#         print(False in list(workerdf['inTast']))
        if ('False' in list(workerdf['inTask'])):
            print(i)
            pair1 = pair(workerdf, workOrderdf, facilitydf, shift)
            if pair1 != None:
                update(pair1[1], workerdf, workOrderdf, pair1[2], facilitydf)


In [741]:
# pair2 = pair(workers, workOrders, facil, 'Morning')
# print(pair2)

In [742]:
# update(pair2[1], workers, workOrders, pair2[2], facil)

In [743]:
assignAll(workers, workOrders, facil, 'Morning')
clear_output()

In [744]:
workers.head(10)

,Equipment Certification(s),Loc,Name,Shifts,TasktimeLeft,inTask
0,"Conveyor, Seperator",Fac1,Andrew,Evening,0,False
1,"Compressor, Electricity",Fac2,Anh,Evening,0,False
2,"Sensor, Security, Networking",Fac1,Bob,Morning,1,True
3,Vehicle,Fac3,Celeste,Evening,0,False
4,"Conveyor, Seperator",Fac5,Jackie,Morning,5,True
5,"Compressor, Electricity",Fac3,Jacob,Morning,9,True
6,"Sensor, Security, Networking",Fac4,Lilly,Evening,0,False
7,Vehicle,Fac1,Marcus,Morning,6,True
8,"Pump, HVAC",Fac5,Mohammed,Evening,0,False
9,"Pump, HVAC",Fac3,Sally,Morning,16,True


In [745]:
facil.head(10)

,Facility,Latitude,Longitude,Maximum Occupacy,workersIn
0,Fac1,29.755537,-95.372003,2,2
1,Fac2,29.716361,-95.409329,2,0
2,Fac3,29.712818,-95.401964,3,2
3,Fac4,29.721943,-95.399359,2,0
4,Fac5,29.717913,-95.401923,3,1


In [746]:
workOrders.head(100)

,Equipment ID,Equipment Type,Facility,Priority(1-5),Submission Timestamp,Time to Complete,Work Order,done,inProgress,newPrior,timeLeft
0,P032,Pump,Fac1,5,2018-09-14 22:12:00,3,1001,False,False,2,0
1,Con391,Conveyer,Fac3,1,2018-09-13 18:30:00,9,1002,False,False,2,0
2,Sep028,Seperator,Fac4,2,2018-09-13 05:41:00,3,1003,False,False,2,0
3,Sen826,Sensor,Fac5,4,2018-09-13 10:20:00,1,1004,False,False,2,0
4,Sec032,Security,Fac1,1,2018-09-13 10:20:00,2,1005,False,False,2,0
5,El087,Electricity,Fac5,3,2018-09-14 08:34:00,2,1006,False,False,2,0
6,Net012,Networking,Fac1,3,2018-09-15 14:52:00,4,1007,False,False,2,0
7,V053,Vehicle,Fac1,4,2018-09-12 12:36:00,4,1008,False,True,8,6
8,HVAC021,HVAC,Fac2,2,2018-09-12 09:23:00,18,1009,False,False,2,0
9,Com471,Compressor,Fac3,5,2018-09-13 18:38:00,9,1010,False,True,8,9


In [346]:
database.init_database("https://hackrice11-ordermanageme-327b0-default-rtdb.firebaseio.com", "/Users/victorkaplan/Desktop/HackRice/HackRice11-OrderManagement/src/DataFiles/database_private_key.json")

ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [732]:
def updatedb():
    workersName = workers.copy()
    workersName.index = workersName['Name']
    workersttodict = workersName.to_dict(orient = 'index')
    database.set_with_dict(workersttodict, '/Workers')
    
    workOrdersName = workOrders.copy()
    workOrdersName['Submission Timestamp'] = workOrdersName['Submission Timestamp'].astype(str)
    workOrdersName.index = workOrdersName['Work Order ']
    workOrdersNametodict = workOrdersName.to_dict(orient = 'index')
    database.set_with_dict(workOrdersNametodict, '/WorkOrders')
    
    facilName = facil.copy()
    facilName.index = facilName['Facility']
    facilNametodict = facilName.to_dict(orient = 'index')
    database.set_with_dict(facilNametodict, '/Facilities')
    
    return None


In [733]:
updatedb()
clear_output()

In [747]:
facil.head()

,Facility,Latitude,Longitude,Maximum Occupacy,workersIn
0,Fac1,29.755537,-95.372003,2,2
1,Fac2,29.716361,-95.409329,2,0
2,Fac3,29.712818,-95.401964,3,2
3,Fac4,29.721943,-95.399359,2,0
4,Fac5,29.717913,-95.401923,3,1
